In [11]:
from mytypes import Queue
import random
import numpy as np

In [7]:
# A class to represent a single customer in an M/D/1 queue simulation.
# Each customer has three attributes:
#
#  - cid: A customer identifier (can be anything, but we will use consecutive integers)
#  - arrival_time: The time at which the customer arrived at the queue
#  - departure_time: The time at which the customer departed the queue
class Customer(object):
    CUSTOMER_ID = 0

    def __init__(self, arrival_time):
        Customer.CUSTOMER_ID += 1
        self.cid = Customer.CUSTOMER_ID
        self.arrival_time = arrival_time
        self.departure_time = None
        
    @property
    def wait(self):
        if self.departure_time is None:
            return None
        else:
            return self.departure_time - self.arrival_time
        
    def __str__(self):
        return "Customer({}, {})".format(self.cid, self.arrival_time)
    
    def __repr__(self):
        return str(self)

In [15]:
# simulate_md1: Simulates an M/D/1 queue.
#
# In an M/D/1 queue que have:
#   
# - Arrivals follow a Markov process (M)
# - The time to service each customer is deterministic (D)
# - There is only one server (1)
#
# The function takes three parameters (plus one optional parameter)
#
# - lambd: The simulation uses an exponential distribution to determine
#          the arrival time of the next customer. This parameters is the
#          lambda parameter to an exponential distribution (specifically,
#          Python's random.expovariate)
# - mu: The rate at which customers are serviced. The larger this value is,
#       the more customers will be serviced per unit of time
# - max_time: The maximum time of the simulation
# - verbosity (optional): Can be 0 (no output), 1 (print state of the queue
#                         at each time), or 2 (same as 1, but also print when
#                         each customer arrives and departs)
#
# The function returns two lists: one with all the customers that were served
# during the simulation, and one with all the customers that were yet to be
# served when the simulation ended.
#

def simulate_md1(lambd, mu, max_time, verbosity = 0):
    md1 = Queue()
    L_array =[]

    # Our return values: the list of customers that have been
    # served, and the list of customers that haven't been served
    served_customers = []
    unserved_customers = []
    
    # The type of simulation we have implemented in this function
    # is known as a "discrete event simulation"
    # (https://en.wikipedia.org/wiki/Discrete_event_simulation), where
    # we simulate a discrete sequence of events: customer arrivals
    # and customer departures. So, we only need to keep track of when 
    # the next arrival and the next departure will take place (because 
    # nothing interesting happens between those two types of events). 
    # Then, in each step of the simulation, we simply advance the 
    # simulation clock to earliest next event. Note that, because
    # we have a single server, this can be easily done with just
    # two variables.

    next_arrival = random.expovariate(lambd)
    next_service = next_arrival + 1/mu
        
    # We initialize the simulation's time to the earliest event:
    # the next arrival time
    t = next_arrival
    
    while t < max_time:

        # Process a new arrival
        if t == next_arrival:
            customer = Customer(arrival_time = t)
            md1.enqueue(customer)

            if verbosity >= 2:
                print("{:10.2f}: Customer {} arrives".format(t, customer.cid))

            next_arrival = t + random.expovariate(lambd)
            
        # The customer at the head of the queue has been served
        if t == next_service:
            done_customer = md1.dequeue()
            done_customer.departure_time = t
            
            served_customers.append(done_customer)

            if verbosity >= 2:
                print("{:10.2f}: Customer {} departs".format(t, done_customer.cid))            
            
            if md1.is_empty():
                # The next service time will be 1/mu after the next arrival
                next_service = next_arrival + 1/mu
            else:
                # We start serving the next customer, so the next service time
                # will be 1/mu after the current time.
                next_service = t + 1/mu
            
        if verbosity >= 1:
            print("{:10.2f}: {}".format(t, "#"*md1.length))
            L_array.append(md1.length)
            print(*L_array)
            
            
        # Advance the simulation clock to the next event
        t = min(next_arrival, next_service)
        
    # Any remaining customers in the queue haven't been served
    while not md1.is_empty():
        unserved_customers.append(md1.dequeue())
        
    return served_customers, unserved_customers, L_array

mu is service rate instead of service, divide 1 hour per customer by 1 customer per hour --> mu =1

In [16]:
simulate_md1(0.9, 1, 100, verbosity=2)

      3.64: Customer 122 arrives
      3.64: #
1
      4.64: Customer 122 departs
      4.64: 
1 0
      4.72: Customer 123 arrives
      4.72: #
1 0 1
      5.26: Customer 124 arrives
      5.26: ##
1 0 1 2
      5.72: Customer 123 departs
      5.72: #
1 0 1 2 1
      5.98: Customer 125 arrives
      5.98: ##
1 0 1 2 1 2
      6.72: Customer 124 departs
      6.72: #
1 0 1 2 1 2 1
      7.33: Customer 126 arrives
      7.33: ##
1 0 1 2 1 2 1 2
      7.72: Customer 125 departs
      7.72: #
1 0 1 2 1 2 1 2 1
      8.58: Customer 127 arrives
      8.58: ##
1 0 1 2 1 2 1 2 1 2
      8.72: Customer 126 departs
      8.72: #
1 0 1 2 1 2 1 2 1 2 1
      9.72: Customer 127 departs
      9.72: 
1 0 1 2 1 2 1 2 1 2 1 0
      9.77: Customer 128 arrives
      9.77: #
1 0 1 2 1 2 1 2 1 2 1 0 1
     10.24: Customer 129 arrives
     10.24: ##
1 0 1 2 1 2 1 2 1 2 1 0 1 2
     10.77: Customer 128 departs
     10.77: #
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1
     11.50: Customer 130 arrives
     11.50: ##
1 0 

1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5
     48.52: Customer 164 arrives
     48.52: ######
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6
     48.56: Customer 159 departs
     48.56: #####
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5
     49.40: Customer 165 arrives
     49.40: ######
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6
     49.51: Customer 166 arrives
     49.51: #######
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 

1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10
     66.69: Customer 188 arrives
     66.69: ###########
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10 11
     67.56: Customer 178 departs
     67.56: ##########
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10 11 10
     67.74: Customer 189 arrives
     67.74: ###########
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 

1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10 11 10 11 12 13 12 13 12 11 10 11 10 9 8 7 8 9 8 7 8 7 6 7 6 5
     82.56: Customer 193 departs
     82.56: ####
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10 11 10 11 12 13 12 13 12 11 10 11 10 9 8 7 8 9 8 7 8 7 6 7 6 5 4
     83.53: Customer 198 arrives
     83.53: #####
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9

1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10 11 10 11 12 13 12 13 12 11 10 11 10 9 8 7 8 9 8 7 8 7 6 7 6 5 4 5 4 3 2 1 0 1 0 1 2 3 4 3 4 5 6 5 6 5 4 5 4 3
     97.07: Customer 209 arrives
     97.07: ####
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5 6 5 6 7 6 7 6 7 8 7 8 9 8 9 10 11 10 9 10 9 10 9 8 9 8 9 8 9 10 11 10 9 10 11 10 11 10 9 10 9 10 11 10 11 10 11 12 13 12 13 12 11 10 11 10 9 8 7 8 9 8 7 8 7 6 7 6 5 4 5 4 3 2 1 0 1 0 1 2 3 4 3 4 5 6 5 6 5 4 5 4 3 4
     97.82: Customer 206 departs
     97.82: ###
1 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 3 2 3 4 5 4 3 4 3 4 3 2 1 2 1 2 1 0 1 2 3 2 3 2 1 2 1 0 1 0 1 0 1 2 1 2 1 2 1 0 1 0 1 2 3 2 3 4 3 4 5 4 5 4 3 4 5 4 5

([Customer(122, 3.64026150009345),
  Customer(123, 4.721986369465651),
  Customer(124, 5.2554300096015485),
  Customer(125, 5.979689952109014),
  Customer(126, 7.33176011065619),
  Customer(127, 8.583593696091707),
  Customer(128, 9.77453568495384),
  Customer(129, 10.24041578239658),
  Customer(130, 11.500049906551059),
  Customer(131, 12.682974560434701),
  Customer(132, 13.520519954233205),
  Customer(133, 13.659481605373179),
  Customer(134, 14.302548523771666),
  Customer(135, 14.346089766432943),
  Customer(136, 14.510260082959107),
  Customer(137, 16.173596295340957),
  Customer(138, 17.582654003642293),
  Customer(139, 20.031038557865685),
  Customer(140, 21.61292045130576),
  Customer(141, 23.363040551749272),
  Customer(142, 23.520082818901116),
  Customer(143, 24.17486219065508),
  Customer(144, 24.506805467103117),
  Customer(145, 26.90516102559755),
  Customer(146, 29.559341728268265),
  Customer(147, 31.571890097335846),
  Customer(148, 35.75486309973879),
  Customer(149,

### n = 100

In [18]:
full_L_array = []
for i in range(10):
    simulate_md1(0.9, 1, 100, verbosity=2)
    print(L_array)
    #full_L_array = np.array(full_L_array, L_array)
print(*full_L_array)

      0.06: Customer 310 arrives
      0.06: #
1
      0.63: Customer 311 arrives
      0.63: ##
1 2
      1.06: Customer 310 departs
      1.06: #
1 2 1
      2.06: Customer 311 departs
      2.06: 
1 2 1 0
      2.18: Customer 312 arrives
      2.18: #
1 2 1 0 1
      3.18: Customer 312 departs
      3.18: 
1 2 1 0 1 0
      4.46: Customer 313 arrives
      4.46: #
1 2 1 0 1 0 1
      4.73: Customer 314 arrives
      4.73: ##
1 2 1 0 1 0 1 2
      5.46: Customer 313 departs
      5.46: #
1 2 1 0 1 0 1 2 1
      5.48: Customer 315 arrives
      5.48: ##
1 2 1 0 1 0 1 2 1 2
      5.60: Customer 316 arrives
      5.60: ###
1 2 1 0 1 0 1 2 1 2 3
      5.84: Customer 317 arrives
      5.84: ####
1 2 1 0 1 0 1 2 1 2 3 4
      6.46: Customer 314 departs
      6.46: ###
1 2 1 0 1 0 1 2 1 2 3 4 3
      7.46: Customer 315 departs
      7.46: ##
1 2 1 0 1 0 1 2 1 2 3 4 3 2
      8.46: Customer 316 departs
      8.46: #
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1
      9.11: Customer 318 arrives
      9.11: #

1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5
     39.66: Customer 344 departs
     39.66: ####
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4
     40.65: Customer 349 arrives
     40.65: #####
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5
     40.66: Customer 345 departs
     40.66: ####
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4
     40.79: Customer 350 arrives
     40.79: #####
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5
     41.66: Customer 346 departs
     41.66: ###

1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 3 4 5 4 5 6 5 4 3 2 3 2 3 4 3 2 3 4 3 4 3 2 1 0 1 2 3 2 1 0 1 2 3 4 5 6 5 4 3
     65.89: Customer 368 departs
     65.89: ##
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 3 4 5 4 5 6 5 4 3 2 3 2 3 4 3 2 3 4 3 4 3 2 1 0 1 2 3 2 1 0 1 2 3 4 5 6 5 4 3 2
     66.34: Customer 371 arrives
     66.34: ###
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 3 4 5 4 5 6 5 4 3 2 3 2 3 4 3 2 3 4 3 4 3 2 1 0 1 2 3 2 1 0 1 2 3 4 5 6 5 4 3 2 3
     66.89: Customer 369 departs
     66.89: ##
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4

1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 3 4 5 4 5 6 5 4 3 2 3 2 3 4 3 2 3 4 3 4 3 2 1 0 1 2 3 2 1 0 1 2 3 4 5 6 5 4 3 2 3 2 3 2 1 0 1 2 1 2 1 0 1 2 1 2 1 0 1 2 1 2 3 2 3 2 3 2 3 4 5 4 5 4 5
     89.19: Customer 385 departs
     89.19: ####
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 3 4 5 4 5 6 5 4 3 2 3 2 3 4 3 2 3 4 3 4 3 2 1 0 1 2 3 2 1 0 1 2 3 4 5 6 5 4 3 2 3 2 3 2 1 0 1 2 1 2 1 0 1 2 1 2 1 0 1 2 1 2 3 2 3 2 3 2 3 4 5 4 5 4 5 4
     90.19: Customer 386 departs
     90.19: ###
1 2 1 0 1 0 1 2 1 2 3 4 3 2 1 2 3 2 1 2 1 2 1 0 1 0 1 2 3 2 3 2 1 2 1 2 3 2 3 4 3 2 1 2 1 2 3 4 3 4 3 4 5 4 3 2 1 2 3 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 5 4 3 4 5 4 5 6 5 4 3 2 3 2 3 4 3 2 3 4 3 4 3 2 1 0 1 2 3 2 1 0 1 2 3 4 5 6 5 4 3 2 3 2 3 2 1 0 1 2 1 2 1 0 1 2 1 2 1 0 1 2

NameError: name 'L_array' is not defined

#### Reference

The code below is adapted from a lectrure example for CMSC 12100/CAPP 30121 at University of Chicago.
The course was instructed by multiple instructors and teaching assistants.

The code below are modified for the use of calculating average system length / average customers in the system (L).

https://classes.cs.uchicago.edu/archive/2017/fall/12100-1/lecture-examples/